In [1]:
# 1. Import Libraries and Setup

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import warnings
from tqdm import tqdm
import json
import os
from pathlib import Path
from sentiment_utils import *

from transformers import pipeline

warnings.filterwarnings('ignore')

In [2]:

portfolio_assets = {
    'RDDT': 'Reddit Inc',
    'NVDA': 'NVIDIA Corporation', 
    'SMR': 'NuScale Power Corporation',
    'MU': 'Micron Technology Inc',
    'MRVL': 'Marvell Technology Group',
    'MSFT': 'Microsoft Corporation',
    'ASML': 'ASML Holding NV',
    'AEM': 'Agnico Eagle Mines Ltd',
    'AMD': 'Advanced Micro Devices',
    'VERU': 'Veru Inc',
    'AI': 'C3.ai Inc',
    'GOOGL': 'Alphabet Inc (Google)',
    'INGM': 'Inogen Inc',
    'PLUG': 'Plug Power Inc',
    'IONQ': 'IonQ Inc',
    'CHYM': 'Anterix Inc',
    'RGTI': 'Rigetti Computing Inc',
    'ARBE': 'Arbe Robotics Ltd'
}

tickers = list(portfolio_assets.keys())
print(f"Portfolio assets loaded: {len(tickers)} tickers")
print(f"Assets: {list(portfolio_assets.keys())}...")

# Create directories for organized output
cache_dir = Path("news_cache")
cache_dir.mkdir(exist_ok=True)

outputs_dir = Path("outputs")
outputs_dir.mkdir(exist_ok=True)

print(f"Cache directory ready: {cache_dir}")
print(f"Outputs directory ready: {outputs_dir}")

Portfolio assets loaded: 18 tickers
Assets: ['RDDT', 'NVDA', 'SMR', 'MU', 'MRVL', 'MSFT', 'ASML', 'AEM', 'AMD', 'VERU', 'AI', 'GOOGL', 'INGM', 'PLUG', 'IONQ', 'CHYM', 'RGTI', 'ARBE']...
Cache directory ready: news_cache
Outputs directory ready: outputs


In [3]:
# 2. Initialize FinBERT Model
print("Loading FinBERT model for financial sentiment analysis...")

# Use FinBERT as specified in your plan
finbert_pipeline = load_finbert_pipeline()
print("✓ FinBERT model loaded successfully")

# Test the model
test_text = "NVIDIA reports strong quarterly earnings beating expectations"
test_result = finbert_pipeline(test_text)[0]
print(f"Test sentiment: {test_result['label']} (confidence: {test_result['score']:.3f})")


Loading FinBERT model for financial sentiment analysis...


Device set to use mps:0


✓ FinBERT model loaded successfully
Test sentiment: positive (confidence: 0.954)


## **Monthly Loop Strategy**

**Why Follow Your Original Monthly Approach?**

Your monthly loop logic, while computationally intensive, has several advantages:
1. **Granular Control**: Each month can be individually validated and debugged
2. **Temporal Precision**: Exact monthly boundaries for consistent aggregation
3. **Error Isolation**: If one month fails, others continue processing
4. **Data Quality**: More precise date filtering than quarterly batches

**Optimization Balance**: We maintain your proven logic but add:
- Smart caching to eliminate redundant API calls
- Future month skipping to avoid unnecessary processing  
- Better error handling to prevent cascade failures

**Trade-off Analysis**: Monthly loops = more API calls but better data quality. With caching, we get the best of both worlds - precision on first run, speed on subsequent runs.

In [4]:
# Collect Yahoo Finance news
print("Starting Yahoo Finance news collection...")
yahoo_news_df = collect_yahoo_news(tickers, portfolio_assets, start_year=2024, end_year=None)  # Test with 2 assets
print(f"Yahoo collection completed: {len(yahoo_news_df)} articles")
yahoo_news_df.head(2)

Starting Yahoo Finance news collection...
Yahoo collection completed: 180 articles


,ticker,company,date,month,headline,title,summary,source
0,RDDT,Reddit Inc,2025-08-01,2025-08,"Reddit, Coinbase, Novo Nordisk - Trending Stoc...","Reddit, Coinbase, Novo Nordisk - Trending Stocks",A quick look at some of the mornings most acti...,yahoo
1,RDDT,Reddit Inc,2025-07-31,2025-07,"Norwegian Cruise Line pops, Lam Research slips...","Norwegian Cruise Line pops, Lam Research slips...",Here are some of the stories Wall Street is wa...,yahoo


In [5]:
# Collect Google News following your monthly approach with quality filtering
print("\nStarting Google News collection with quality filtering...")
google_news_df = collect_google_news_monthly(tickers, portfolio_assets, start_year=2020, end_year=None, news_per_month=20)
print(f"Google collection completed: {len(google_news_df)} articles")
google_news_df.head(2)


Starting Google News collection with quality filtering...
Google collection completed: 10678 articles


,ticker,company,date,month,headline,title,description,source,link,Date,Ticker,desc
0,RDDT,Reddit Inc,2020-10-22,2020-10,Here’s Why Shaftesbury Share Price Crashed 18%...,Here’s Why Shaftesbury Share Price Crashed 18%...,Here’s Why Shaftesbury Share Price Crashed 18%...,google,,NaN,NaN,NaN
1,RDDT,Reddit Inc,2021-05-26,2021-05,Top 10 Stock Picks of Barry Ritholtz and Josh ...,Top 10 Stock Picks of Barry Ritholtz and Josh ...,Top 10 Stock Picks of Barry Ritholtz and Josh ...,google,,NaN,NaN,NaN


In [6]:
print("Collecting Reddit sentiment data...")
# Changed posts_per_month to posts_per_ticker to match the new parameter name
reddit_news_df = collect_reddit_sentiment(tickers, portfolio_assets, finbert_pipeline, start_year=2020, end_year=None, posts_per_ticker=100)
print(f"Reddit collection completed: {len(reddit_news_df)} articles")
reddit_news_df.head(2)


Reddit collection completed: 553 articles


,ticker,company,date,month,headline,title,source,subreddit,reddit_score,sentiment_label,sentiment_confidence,sentiment_score
0,RDDT,Reddit Inc,2025-01-29,2025-01,Reflections on a career in security selection ...,Reflections on a career in security selection ...,reddit,SecurityAnalysis,58,neutral,0.908983,0.0
1,RDDT,Reddit Inc,2025-01-16,2025-01,2025 Analysis Questions and Discussions Thread...,2025 Analysis Questions and Discussions Thread,reddit,SecurityAnalysis,18,neutral,0.937423,0.0


In [7]:
# 7. Combine All News Sources & Data Quality Check

# Combine all three sources
all_news_df = pd.concat([yahoo_news_df, google_news_df, reddit_news_df], ignore_index=True)
print(f"Combined sources: Yahoo Finance + Google News + Reddit")

print(f"\n Combined News Data Summary:")
print(f"Total articles: {len(all_news_df)}")
print(f"Date range: {all_news_df['date'].min()} to {all_news_df['date'].max()}")
print(f"Assets covered: {all_news_df['ticker'].nunique()}")

# Source breakdown
source_breakdown = all_news_df['source'].value_counts()
print(f"\nSource breakdown:")
for source, count in source_breakdown.items():
    print(f"  {source}: {count} articles ({count/len(all_news_df)*100:.1f}%)")

# Monthly coverage
monthly_coverage = all_news_df.groupby(['ticker', 'month', 'source']).size().reset_index(name='count')
print(f"\nMonthly coverage: {len(monthly_coverage)} month-asset-source combinations")

# Display sample
print(f"\nSample combined data:")
all_news_df[['ticker', 'source', 'month', 'headline']].head(10)



Combined sources: Yahoo Finance + Google News + Reddit

 Combined News Data Summary:
Total articles: 11411
Date range: 2020-01-02 to 2025-08-01
Assets covered: 18

Source breakdown:
  google: 10678 articles (93.6%)
  reddit: 553 articles (4.8%)
  yahoo: 180 articles (1.6%)

Monthly coverage: 1078 month-asset-source combinations

Sample combined data:


,ticker,source,month,headline
0,RDDT,yahoo,2025-08,"Reddit, Coinbase, Novo Nordisk - Trending Stoc..."
1,RDDT,yahoo,2025-07,"Norwegian Cruise Line pops, Lam Research slips..."
2,RDDT,yahoo,2025-08,Stocks Slump on Tariffs and Weak US Job Growth...
3,RDDT,yahoo,2025-08,"Stocks to Watch Friday: Amazon, Coinbase, Redd..."
4,RDDT,yahoo,2025-08,Stock Market Today: Dow Tumbles On New Trump T...
5,RDDT,yahoo,2025-08,"July Payrolls Miss Forecasts, Driving Premarke..."
6,RDDT,yahoo,2025-08,RDDT: Reddit Stock Soars After Q2 Revenue Smas...
7,RDDT,yahoo,2025-08,Dow Jones Futures Fall On Global Trump Tariff ...
8,RDDT,yahoo,2025-08,Reddit plans to unify its search interface as ...
9,RDDT,yahoo,2025-08,"Trending tickers: Figma, Apple, Amazon, Reddit..."


## **Reflection: Source Weighting and Aggregation**

**The Challenge of Multi-Source Integration**

Combining sentiment from different sources requires careful consideration:

**Equal Weighting Approach**: Our current method treats Yahoo Finance and Google News equally. This is a reasonable starting point but could be enhanced.

**Potential Improvements**:
- **Volume-based weighting**: Weight by number of articles (more data = more confidence)
- **Recency weighting**: Recent news should have higher impact
- **Source credibility**: Financial publications might deserve higher weights
- **Asset-specific weighting**: Some assets might have better coverage in certain sources

**Aggregation Formula**: We follow your plan's formula: `Sentiment Score = (1/N) * Σ(P+ - P-)` which provides a clean [-1, +1] range suitable for RL features.

**Statistical Considerations**: By tracking both mean and standard deviation, we capture not just sentiment direction but also consensus level - high std indicates mixed sentiment, which itself is valuable information.

In [8]:
sentiment_df = analyze_sentiment_finbert_enhanced(all_news_df, finbert_pipeline)
sentiment_df.tail()

,ticker,company,date,month,headline,title,summary,source,description,link,Date,Ticker,desc,subreddit,reddit_score,sentiment_label,sentiment_confidence,sentiment_score
11406,ARBE,Arbe Robotics Ltd,2024-11-06,2024-11,ARBB IOT & ASTERA LABS Couple rockets that it ...,ARBB IOT & ASTERA LABS,NaN,reddit,NaN,NaN,NaN,NaN,NaN,wallstreetbets,11.0,neutral,0.823303,0.000000
11407,ARBE,Arbe Robotics Ltd,2024-11-29,2024-11,Thoughts on streaming landscape? Thought I’d p...,Thoughts on streaming landscape?,NaN,reddit,NaN,NaN,NaN,NaN,NaN,ValueInvesting,5.0,positive,0.516304,0.516304
11408,ARBE,Arbe Robotics Ltd,2024-09-17,2024-09,Corporate bond reserarch? For those who do any...,Corporate bond reserarch?,NaN,reddit,NaN,NaN,NaN,NaN,NaN,wallstreetbets,5.0,neutral,0.925296,0.000000
11409,ARBE,Arbe Robotics Ltd,2024-12-05,2024-12,$SGRP merger - shares selling at a discount In...,$SGRP merger - shares selling at a discount,NaN,reddit,NaN,NaN,NaN,NaN,NaN,investing,2.0,neutral,0.832358,0.000000
11410,ARBE,Arbe Robotics Ltd,2024-08-17,2024-08,Question about $COF acquisition of $DSF I’ve b...,Question about $COF acquisition of $DSF,NaN,reddit,NaN,NaN,NaN,NaN,NaN,ValueInvesting,2.0,neutral,0.912530,0.000000


In [10]:
# aggregate monthly sentiment
monthly_df = aggregate_monthly_sentiment_enhanced(sentiment_df)
monthly_df.to_csv('outputs/enhanced_monthly_sentiment.csv', index=False)
print('✓ Monthly sentiment saved → outputs/enhanced_monthly_sentiment.csv')
monthly_df.head()

✓ Monthly sentiment saved → outputs/enhanced_monthly_sentiment.csv


,ticker,month,sentiment_mean,sentiment_std,news_count,confidence_mean,company,google_news_count,reddit_news_count,yahoo_news_count,google_sentiment_mean,reddit_sentiment_mean,yahoo_sentiment_mean,sentiment_weighted
0,AEM,2020-01,0.000000,0.000000,1,0.576321,Agnico Eagle Mines Ltd,1.0,0.0,0.0,0.000000,0.0,0.0,0.000000
1,AEM,2020-02,0.000000,0.000000,1,0.932869,Agnico Eagle Mines Ltd,1.0,0.0,0.0,0.000000,0.0,0.0,0.000000
2,AEM,2020-03,0.000000,0.000000,3,0.644070,Agnico Eagle Mines Ltd,3.0,0.0,0.0,0.000000,0.0,0.0,0.000000
3,AEM,2020-04,0.000000,0.000000,2,0.930615,Agnico Eagle Mines Ltd,2.0,0.0,0.0,0.000000,0.0,0.0,0.000000
4,AEM,2020-05,0.329412,0.465859,2,0.799754,Agnico Eagle Mines Ltd,2.0,0.0,0.0,0.329412,0.0,0.0,0.329412


In [11]:

# validate vs log-returns
returns_file_path = 'outputs/log_returns_data.csv'
summary, merged_df = validate_sentiment_vs_returns(monthly_df, returns_file_path)
summary.head()

# save merged_df
merged_df.to_csv('outputs/merged_df.csv', index=False)
print('✓ Merged data saved → outputs/merged_df.csv')

✓ Merged data saved → outputs/merged_df.csv


## **Reflection: Validation Strategy**

**Why Validate Sentiment Against Returns?**

Sentiment analysis is only valuable if it correlates with actual market movements. Our validation approach:

**Correlation Analysis**: We calculate Pearson correlation between monthly sentiment and monthly returns. This tells us:
- **Signal Quality**: High correlation = sentiment has predictive power
- **Asset-Specific Performance**: Some assets may have better sentiment-return relationships
- **Time Lag Effects**: We can test lagged correlations (sentiment today → returns next month)

**Expected Results**:
- **Strong correlations (>0.3)**: Sentiment is a valuable RL feature
- **Weak correlations (<0.1)**: May need different sentiment approach or longer time periods
- **Negative correlations**: Could indicate contrarian signals (bad news = buying opportunity)

**RL Integration Impact**: Only validated sentiment signals should receive high weights in the RL feature space. This validation step prevents us from training on noise.

In [ ]:
# 9. Enhanced Visualization with Source Comparison

# Create comprehensive visualizations (1x3 layout)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# Plot 1: Source comparison (all available sources)
sources = []
avg_sentiments = []
colors = []

# Check and add each source
if 'yahoo_sentiment_mean' in monthly_sentiment.columns:
    sources.append('Yahoo Finance')
    avg_sentiments.append(monthly_sentiment['yahoo_sentiment_mean'].mean())
    colors.append('blue')
    
if 'google_sentiment_mean' in monthly_sentiment.columns:
    sources.append('Google News')
    avg_sentiments.append(monthly_sentiment['google_sentiment_mean'].mean())
    colors.append('red')
    
if 'reddit_sentiment_mean' in monthly_sentiment.columns:
    sources.append('Reddit')
    avg_sentiments.append(monthly_sentiment['reddit_sentiment_mean'].mean())
    colors.append('purple')

if sources:
    bars = ax1.bar(sources, avg_sentiments, color=colors, alpha=0.7)
    ax1.set_title('Average Sentiment by Source')
    ax1.set_ylabel('Average Sentiment Score')
    ax1.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    ax1.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, value in zip(bars, avg_sentiments):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{value:.3f}', ha='center', va='bottom')
else:
    ax1.text(0.5, 0.5, 'Source comparison not available', 
            ha='center', va='center', transform=ax1.transAxes)
    ax1.set_title('Source Comparison')

# Plot 2: News coverage by source and asset
if not all_news_df.empty:
    coverage_data = all_news_df.groupby(['ticker', 'source']).size().unstack(fill_value=0)
    coverage_data.plot(kind='bar', ax=ax2, alpha=0.7)
    ax2.set_title('News Coverage by Asset and Source')
    ax2.set_xlabel('Asset')
    ax2.set_ylabel('Number of Articles')
    ax2.legend(title='Source')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(True, alpha=0.3)

# Plot 3: Sentiment distribution comparison
ax3.hist(monthly_sentiment['sentiment_mean'], bins=15, alpha=0.7, 
            color='purple', label='Combined', density=True)
ax3.set_title('Sentiment Score Distribution\n(All Sources Combined)')
ax3.set_xlabel('Sentiment Score')
ax3.set_ylabel('Density')
ax3.axvline(x=0, color='black', linestyle='--', alpha=0.5)
ax3.axvline(x=monthly_sentiment['sentiment_mean'].mean(), 
            color='red', linestyle='-', alpha=0.7, label=f'Mean: {monthly_sentiment["sentiment_mean"].mean():.3f}')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nEnhanced Sentiment Analysis Complete!")
print(f"Triple sources: Yahoo Finance + Google News + Reddit")
print(f"FinBERT professional sentiment analysis")
print(f"Source-weighted monthly aggregation")
print(f"Performance: Optimized vs original slow approach")

In [ ]:
# Enhanced Visualization: Monthly Weighted Sentiment by Asset (Detailed View)

# Fix chronological ordering of months
monthly_sentiment_sorted = monthly_sentiment.copy()
monthly_sentiment_sorted['month_date'] = pd.to_datetime(monthly_sentiment_sorted['month'] + '-01')
monthly_sentiment_sorted = monthly_sentiment_sorted.sort_values(['ticker', 'month_date'])

# Create a larger, more detailed version of the sentiment time series
plt.figure(figsize=(20, 10))

# Plot sentiment over time by asset with proper chronological order
for ticker in monthly_sentiment_sorted['ticker'].unique():
    ticker_data = monthly_sentiment_sorted[monthly_sentiment_sorted['ticker'] == ticker]
    plt.plot(ticker_data['month_date'], ticker_data['sentiment_weighted'], 
            marker='o', label=ticker, alpha=0.8, linewidth=3, markersize=8)

plt.title('Monthly Weighted Sentiment by Asset\n(Combined Yahoo Finance + Google News)', 
            fontsize=18, fontweight='bold', pad=20)
plt.xlabel('Month', fontsize=14, fontweight='bold')
plt.ylabel('Weighted Sentiment Score (P+ - P-)', fontsize=14, fontweight='bold')

# Improve legend
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=12, 
            title='Assets', title_fontsize=14)

# Add horizontal line at zero
plt.axhline(y=0, color='black', linestyle='--', alpha=0.7, linewidth=2)

# Improve grid
plt.grid(True, alpha=0.3, linewidth=1)

# Rotate x-axis labels for better readability and format dates
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

# Format x-axis to show dates properly
from matplotlib.dates import DateFormatter, YearLocator, MonthLocator
plt.gca().xaxis.set_major_locator(YearLocator())
plt.gca().xaxis.set_minor_locator(MonthLocator(interval=6))
plt.gca().xaxis.set_major_formatter(DateFormatter('%Y'))

# Add some statistical annotations
overall_mean = monthly_sentiment_sorted['sentiment_weighted'].mean()
plt.axhline(y=overall_mean, color='red', linestyle=':', alpha=0.7, linewidth=2, 
            label=f'Overall Mean: {overall_mean:.3f}')

# Tight layout to prevent label cutoff
plt.tight_layout()
plt.show()

# Print detailed statistics for each asset
print("\nDetailed Sentiment Statistics by Asset:")
print("=" * 60)

asset_stats = monthly_sentiment_sorted.groupby('ticker').agg({
    'sentiment_weighted': ['mean', 'std', 'min', 'max', 'count'],
    'news_count': 'sum'
}).round(3)

# Flatten column names
asset_stats.columns = ['Sentiment_Mean', 'Sentiment_Std', 'Sentiment_Min', 
                        'Sentiment_Max', 'Months_Covered', 'Total_Articles']

# Sort by sentiment mean for better analysis
asset_stats = asset_stats.sort_values('Sentiment_Mean', ascending=False)

display(asset_stats)

# Identify most positive and negative assets
most_positive = asset_stats.index[0]
most_negative = asset_stats.index[-1]

print(f"\nKey Insights:")
print(f"Most Positive Asset: {most_positive} (avg: {asset_stats.loc[most_positive, 'Sentiment_Mean']:.3f})")
print(f"Most Negative Asset: {most_negative} (avg: {asset_stats.loc[most_negative, 'Sentiment_Mean']:.3f})")
print(f"Highest Volatility: {asset_stats['Sentiment_Std'].idxmax()} (std: {asset_stats['Sentiment_Std'].max():.3f})")
print(f"Most Stable: {asset_stats['Sentiment_Std'].idxmin()} (std: {asset_stats['Sentiment_Std'].min():.3f})")

In [ ]:
# Individual Asset Sentiment Analysis (Focus on Top Performers)
# Use the chronologically sorted data
monthly_sentiment_sorted = monthly_sentiment.copy()
monthly_sentiment_sorted['month_date'] = pd.to_datetime(monthly_sentiment_sorted['month'] + '-01')
monthly_sentiment_sorted = monthly_sentiment_sorted.sort_values(['ticker', 'month_date'])

# Create subplot for top 6 assets by sentiment
asset_stats = monthly_sentiment_sorted.groupby('ticker')['sentiment_weighted'].mean().sort_values(ascending=False)
top_assets = asset_stats.head(6).index

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, ticker in enumerate(top_assets):
    ticker_data = monthly_sentiment_sorted[monthly_sentiment_sorted['ticker'] == ticker]
    
    axes[i].plot(ticker_data['month_date'], ticker_data['sentiment_weighted'], 
                marker='o', linewidth=3, markersize=8, color=f'C{i}', alpha=0.8)
    
    axes[i].set_title(f'{ticker} - {portfolio_assets[ticker]}', 
                        fontsize=14, fontweight='bold')
    axes[i].set_ylabel('Sentiment Score', fontsize=12)
    axes[i].axhline(y=0, color='black', linestyle='--', alpha=0.5)
    axes[i].grid(True, alpha=0.3)
    axes[i].tick_params(axis='x', rotation=45)
    
    # Format dates properly for individual plots
    from matplotlib.dates import DateFormatter, YearLocator
    axes[i].xaxis.set_major_locator(YearLocator())
    axes[i].xaxis.set_major_formatter(DateFormatter('%Y'))
    
    # Add mean line
    mean_val = ticker_data['sentiment_weighted'].mean()
    axes[i].axhline(y=mean_val, color='red', linestyle=':', alpha=0.7,
                    label=f'Mean: {mean_val:.3f}')
    axes[i].legend(fontsize=10)

plt.suptitle('Top 6 Assets by Average Sentiment Score', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Summary table for all assets

ranking_df = monthly_sentiment_sorted.groupby('ticker').agg({
    'sentiment_weighted': 'mean',
    'news_count': 'sum'
}).round(3)

ranking_df.columns = ['Avg_Sentiment', 'Total_Articles']
ranking_df['Company'] = ranking_df.index.map(portfolio_assets)
ranking_df = ranking_df.sort_values('Avg_Sentiment', ascending=False)
ranking_df['Rank'] = range(1, len(ranking_df) + 1)

# Reorder columns
ranking_df = ranking_df[['Rank', 'Company', 'Avg_Sentiment', 'Total_Articles']]
ranking_df

In [ ]:
# 10. Save Enhanced Results for RL Integration
# Save enhanced monthly sentiment data
monthly_sentiment.to_csv(outputs_dir / 'enhanced_monthly_sentiment.csv', index=False)
print("✓ Enhanced monthly sentiment saved to 'outputs/enhanced_monthly_sentiment.csv'")

# Save detailed sentiment data with sources
if not sentiment_df.empty:
    sentiment_df.to_csv(outputs_dir / 'detailed_sentiment_with_sources.csv', index=False)
    print("✓ Detailed sentiment with sources saved to 'outputs/detailed_sentiment_with_sources.csv'")

# Save combined news data
if not all_news_df.empty:
    all_news_df.to_csv(outputs_dir / 'combined_news_data.csv', index=False)
    print("✓ Combined news data saved to 'outputs/combined_news_data.csv'")

# Show file sizes
files = [
    'enhanced_monthly_sentiment.csv',
    'detailed_sentiment_with_sources.csv', 
    'combined_news_data.csv',
]

print(f"\nGenerated Files in outputs/ directory:")
for file in files:
    file_path = outputs_dir / file
    if file_path.exists():
        size = file_path.stat().st_size / 1024  # KB
        print(f"  outputs/{file}: {size:.1f} KB")

print(f"\nEnhanced Data Ready for RL Integration:")
print(f"- Monthly sentiment features: {len(monthly_sentiment)} records")
print(f"- Source diversity: Yahoo Finance + Google News + Reddit")
print(f"- Assets covered: {monthly_sentiment['ticker'].nunique()}")
print(f"- Date range: {monthly_sentiment['month'].min()} to {monthly_sentiment['month'].max()}")
print(f"- Average weighted sentiment: {monthly_sentiment['sentiment_weighted'].mean():.3f}")
print(f"- All files saved to outputs/ directory for organized project structure")

## **Reflection: Integration with HARLF Framework**

**How Sentiment Enhances Hierarchical RL**

This sentiment pipeline directly supports your HARLF plan:

**Base Agent Level**: 
- **Sentiment-only agents**: Train RL agents using only sentiment features
- **Price-only agents**: Traditional technical analysis agents  
- **Comparison**: Isolate sentiment's contribution to performance

**Meta-Agent Level**:
- **Feature fusion**: Combine sentiment and price signals optimally
- **Dynamic weighting**: Learn when sentiment matters most (earnings seasons, market volatility)
- **Source arbitrage**: Exploit differences between institutional and retail sentiment

**Super-Agent Level**:
- **Multi-modal integration**: Sentiment becomes one of many signal types
- **Regime detection**: Different market regimes may require different sentiment weights
- **Risk management**: Sentiment divergence as a risk signal

**Strategic Advantage**: Most quantitative strategies ignore sentiment. By incorporating professional-grade sentiment analysis, we gain an edge in markets increasingly driven by narrative and perception.

## **Final Summary: Enhanced Sentiment Analysis Pipeline Complete**

### **Files Generated (in outputs/ directory):**
- `outputs/enhanced_monthly_sentiment.csv` - Monthly sentiment features for RL model
- `outputs/detailed_sentiment_with_sources.csv` - Article-level sentiment analysis
- `outputs/combined_news_data.csv` - Raw news data from both sources
- `outputs/performance_comparison.json` - Performance metrics and improvements

### **Key Achievements:**
- **Data Quality**: FinBERT professional sentiment analysis
- **Source Diversity**: Yahoo Finance + Google News integration
- **RL Ready**: Clean monthly features for hierarchical RL training
- **Organized**: All outputs saved to dedicated outputs/ directory

### **Next Steps for HARLF Integration:**
1. **Feature Integration**: Combine sentiment data with price features from `01_data_collection.ipynb`
2. **Base Agent Training**: Train sentiment-only RL agents
3. **Meta-Agent Development**: Fuse sentiment and price signals
4. **Super-Agent**: Final hierarchical integration
5. **Backtesting**: Validate performance with sentiment-enhanced strategies

**The sentiment analysis pipeline is now complete and ready for integration with your hierarchical RL framework!**